# RAG Integration with OpenWebUI
# 🌐 OpenWebUI

**OpenWebUI** is a fully open-source, self-hosted user interface designed to interact with Large Language Models (LLMs) like OpenAI, LLaMA, Mistral, Gemini, DeepSeek, and more — all through an intuitive, chat-style experience. It's built for developers, researchers, and organizations seeking a privacy-focused, extensible alternative to proprietary AI interfaces.

---

## 🚀 Key Highlights

- ✅ **Local-first LLM Interface** – No vendor lock-in.
- 💬 **Multi-Model Chat Interface** – Use multiple LLMs with tabbed conversations.
- 🧠 **Memory & Context Management** – Long-term memory per user, per thread.
- 📚 **RAG Support** – Easily extend with custom Retrieval-Augmented Generation (RAG) backends.
- 📦 **Plugin System** – Add custom filters, pipes, and integrations.
- 🔒 **Private & Secure** – Full control over your data and infrastructure.
- 🌐 **Multi-User Support** – Teams can use the same instance with role-based controls.
- 🎨 **Responsive UI** – Optimized for desktop, mobile, and tablet.
- 🛠️ **Dev-Friendly APIs** – Extend or embed easily in your own systems.

---

## 📘 What Can You Do with OpenWebUI?

- Run **ChatGPT-style conversations** on your own infrastructure.
- Integrate **custom RAG backends** for domain-specific document Q&A.
- Switch between LLMs like GPT-4, Mistral, LLaMA, or DeepSeek on the fly.
- Customize conversations with **system instructions, memory, and tools**.
- Deploy internally in organizations for **private, secure AI access**.
- Build **custom plugins** for translation, summarization, search, or any LLM task.

---

## 🏗️ Architecture Overview

OpenWebUI is built using a modular architecture with the following layers:

1. **Frontend (WebUI)**
   - React-based, responsive design.
   - Chat interface, settings, and plugin management.
2. **Backend (API Layer)**
   - Manages session, LLM calls, memory, and plugins.
   - Written in Python with FastAPI.
3. **Plugins**
   - Filters (pre-processing)
   - Pipes (mid-processing like RAG)
   - Outlets (post-processing)
4. **LLM Adapter**
   - Interfaces with local/remote LLMs (Ollama, OpenAI, LM Studio, etc.)

---

## 📦 Supported LLM Backends

- 🌐 OpenAI (ChatGPT, GPT-4)
- 🦙 Ollama (LLaMA, Mistral, DeepSeek, etc.)
- 🔮 LM Studio
- 🪄 Local GGUF models
- 🧠 Any HTTP API with OpenAI-compatible schema

---

## 📁 Folder Structure

---

## 🔌 Plugin Ecosystem

OpenWebUI supports a plugin-based architecture via:

- **Filters** – Intercept input for moderation, formatting, rewriting.
- **Pipes** – Inject RAG or dynamic content.
- **Outlets** – Post-process output (e.g., rewrite, translate).
- **Inlets** – Customize query routing logic.

Plugins are written in Python and dropped into the `plugins/` directory. They are auto-loaded and configurable via the UI.

---

## 🧪 Example Use Cases

- 🧑‍🏫 **Educational Assistant** – Connect to courseware and documents via RAG.
- 🏢 **Enterprise Chatbot** – Hosted internally, supports fine-tuned LLMs.
- 📖 **Document QA System** – Embed custom PDF Q&A workflows.
- 🔧 **Dev Tooling** – Automate code generation, debugging, and explanations.
- 🌍 **Multilingual Translator** – Use multiple LLMs for real-time translation.

---

## ⚙️ Installation (Quick Start)

```bash
git clone https://github.com/open-webui/open-webui
cd open-webui
docker-compose up --build 
```

Alternatively, follow the below link to install the OpenWebUI
https://docs.openwebui.com/




# Follow the below steps to do the integration
## Step-1 Open OpenWebUI and Then Go to Setting
!["Setting"](images/Step-1.png)

## Step-2: Further, Select Admin Settings from the setting
!["Admin Settings"](images/Step-2.png)

## Setp-3: Go to Functions and then click on the "Create Function" button.
!["Create FUnction"](images/Step-3.png)

## Step-4: Provide a function name and description for your functon.
Further copy and past the below code in the coding window.
!["Functions"](images/Step-4.png)

## Use your RAG API in OpenWebUI
Inside the model selection windows (which you chat window), select the model name to start your chat.
!["Utilize the RAG Workflow"](images/Step-5.png)

In [ ]:
"""
title: RAG Integration
author: dr-avinash
author_url: https://github.com/dr-avinash
version: 1.0.0
required_open_webui_version: 0.3.30
"""

from typing import List, Union, Generator, Iterator
from pydantic import BaseModel, Field
import requests


class Pipe:
    ###############################################################################################
    class Valves(BaseModel):
        rag_api_url: str = Field(
            default="http://192.168.0.106:8000/rag",  # Replace with your actual RAG endpoint
            description="RAG API endpoint",
        )
        enabled: bool = Field(default=True, description="Enable or disable RAG mode")
    ###############################################################################################
    def __init__(self):
        self.type = "manifold"
        self.id = "rag_integration"
        self.name = "RAG"
        self.valves = self.Valves()
    ###############################################################################################
    def pipes(self) -> List[dict]:
        if self.valves.enabled:
            return [{"id": "rag", "name": "-Demo"}]
        return []
    ###############################################################################################
    def pipe(self, body: dict, results=None) -> Union[str, Generator[str, None, None]]:
        user_input = self._extract_user_input(body)
        if not user_input:
            yield "No query provided to RAG system"
            return

        print(f"RAG Query: {user_input}")
        try:
            response = requests.post(
                self.valves.rag_api_url, json={"query": user_input}, timeout=15
            )
            response.raise_for_status()
            result = response.json()
            final_text = result.get("response", "⚠️ No response from RAG API")

            # 🔁 Stream word by word
            # for word in final_text.split():
            #     yield word + " "
            # 🔁 Stream character by character
            for char in final_text:
                yield char + " "
        except Exception as e:
            error_msg = f"❌ Error contacting RAG API: {str(e)}"
            print(error_msg)
            yield error_msg

    ###############################################################################################
    def _extract_user_input(self, body: dict) -> str:
        messages = body.get("messages", [])
        if messages:
            last_message = messages[-1]
            if isinstance(last_message.get("content"), list):
                for item in last_message["content"]:
                    if item["type"] == "text":
                        return item["text"]
            else:
                return last_message.get("content", "")
        return ""
    ###############################################################################################
